In [144]:
import xarray as xr
import plotly.express as px
import plotly.subplots

In [141]:
data = xr.open_dataset("../1_Forcing/hot_dataset.zarr", engine="zarr")
data.load()

<xarray.Dataset> Size: 2MB
Dimensions:                    (time: 659, latitude: 1, longitude: 1, depth: 8,
                                frac: 5, layer: 3)
Coordinates:
  * depth                      (depth) int64 64B 50 100 150 200 250 300 350 400
  * frac                       (frac) float64 40B 0.2 0.5 1.0 2.0 5.0
  * latitude                   (latitude) float64 8B 22.75
  * layer                      (layer) float32 12B 1.0 2.0 3.0
  * longitude                  (longitude) int64 8B -158
  * time                       (time) datetime64[ns] 5kB 1988-10-31 ... 2022-...
Data variables: (12/37)
    bsal                       (time, latitude, longitude, depth) float64 42kB ...
    carb_migrant               (frac, time, latitude, longitude, depth) float64 211kB ...
    carb_resident              (frac, time, latitude, longitude, depth) float64 211kB ...
    chl                        (time, latitude, longitude, depth) float64 42kB ...
    cmems_T                    (time, layer, latitude, longitude) float32 8kB ...
    cmems_U                    (time, layer, latitude, longitude) float32 8kB ...
    ...                         ...
    sigma                      (time, latitude, longitude, depth) float64 42kB ...
    temp                       (time, latitude, longitude, depth) float64 42kB ...
    theta                      (time, latitude, longitude, depth) float64 42kB ...
    wwt_migrant                (frac, time, latitude, longitude, depth) float64 211kB ...
    wwt_resident               (frac, time, latitude, longitude, depth) float64 211kB ...
    xmiss                      (time, latitude, longitude, depth) float64 42kB ...
Attributes:
    Conventions:       CF-1.11
    history:           Created on 2022-09-28
    institution:       CLS
    references:        http://www.cls.fr; http://www.seapodym.eu
    source:            SEAPODYM-LMTL 3.0.0
    subset:datasetId:  cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_202211
    subset:date:       2024-06-11T23:54:54.527Z
    subset:productId:  GLOBAL_MULTIYEAR_BGC_001_033
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    title:             Global ocean low and mid trophic levels biomass conten...

In [142]:
seapodym_zoo = data.cmems_zooc[:, 0, 0].dropna("time")

obs_zoo = data.dwt_resident.sum(["frac"]).mean("depth")
obs_zoo = obs_zoo.where(obs_zoo > 0).dropna("time")[:, 0, 0]

## Comparison


In [145]:
zoo_line = xr.merge([seapodym_zoo, obs_zoo]).to_dataframe().drop(columns=["latitude", "longitude"])

diff_zoo = ((obs_zoo - seapodym_zoo) / obs_zoo) * 100
diff_zoo.name = "diff_zoo"
diff_zoo = diff_zoo.to_dataframe().reset_index().drop(columns=["latitude", "longitude"])
diff_zoo = diff_zoo.resample("3ME", on="time").mean()["diff_zoo"]

In [192]:
fig = plotly.subplots.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0, row_heights=[3, 1])

hot_zoo = zoo_line["cmems_zooc"].dropna()
fig.add_scatter(x=hot_zoo.index, y=hot_zoo, marker={"color": "blue"}, row=1, col=1, name="HOT station")
cmems_zoo = zoo_line["dwt_resident"].dropna()
fig.add_scatter(x=cmems_zoo.index, y=cmems_zoo, marker={"color": "red"}, row=1, col=1, name="Seapodym")


fig.add_bar(x=diff_zoo.index, y=diff_zoo, row=2, col=1, marker={"color": diff_zoo}, name="Difference")

fig.update_xaxes({"title": "Time"}, row=2)
fig.update_layout({"title": "Zooplankton comparison between HOT station and Seapodym model"})
fig.update_yaxes({"title": "Zooplankton (g/m2)"}, row=1)
fig.update_yaxes({"title": "Difference in % of HOT zooplankton"}, row=2)
fig.show()

In [196]:
with open("HOT_comparison.html", "w") as f:
    f.write(fig.to_html())